In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


%matplotlib inline
plt.style.use('fivethirtyeight')          #Used for replicating graph styles from fivethirtyeight.com

from keras.models import load_model

from binance.client import Client

In [9]:
#Loading the trained model
model = load_model('Bitcoin_model.h5')

#Summarize the model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 4, 20)             1760      
_________________________________________________________________
lstm_11 (LSTM)               (None, 40)                9760      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 41        
Total params: 11,561
Trainable params: 11,561
Non-trainable params: 0
_________________________________________________________________


In [10]:
#loading the data from Binance API
client = Client("3NMyXJiv5LqOR9zDrmuCyOnEEVBuV0XX7CDpOwzcZHrgocW0VoR2GJKxU8nRcJPO","tWqMqmODIu6hKqLavQErL177IlVRRNIwy3X43n7L33vlqLjcuWVWHw4FV4M5hatK")
check = client.get_klines(symbol='BTCUSDT',interval = Client.KLINE_INTERVAL_1MINUTE)

check[499]

[1617757800000,
 '57596.20000000',
 '57613.82000000',
 '57588.44000000',
 '57590.82000000',
 '28.91924800',
 1617757859999,
 '1665845.72440510',
 839,
 '12.94733000',
 '745793.53723415',
 '0']

In [11]:
price = np.array([float(check[i][4]) for i in range(500)])
price = price.reshape(500,1)

scaler = StandardScaler()
scaler.fit(price[:374])

price = scaler.transform(price)

In [12]:
index = [496,497,498,499]

candles = scaler.transform(np.array([float(check[i][4]) for i in index]).reshape(-1,1))

model_feed = candles.reshape(1,4,1)

scaler.inverse_transform(model.predict(model_feed)[0])[0]

57663.992

## Trading

In [ ]:
symbol = "BTCUSDT"   #Code of cryptocurrency
quantity = '0.05'    #quantity to trade

order = False
index = [496,497,498,499]

while True:
    price = client.get_recent_trades(symbol=symbol)
    candle = client.get_klines(symbol=symbol,interval=Client.KLINE_INTERVAL_1MINUTE)
    candles = scaler.transform(np.array([float(candle[i][4]) for i in index]).reshape(-1,1))
    model_feed = candles.reshape(1,4,1)
    
    
    if order == False and float(price[len(price)-1]['price']) < float(scaler.inverse_transform(model.predict(model_feed)[0])[0]):
        
        #client.order_market_buy(symbol=symbol,quantity = quantity)
        order = True
        buy_price = client.get_order_book(symbol=symbol)['asks'][0][0]
        print('Buy @Market Price : ',float(buy_price),'Timestamp : ',str(datetime.now()))
        
    elif order == True and float(price[len(price)-1]['price'])-float(buy_price) >=10:
        
        #client.order_market_sell(symbol=symbol, quantity=quantity)
        order = False
        sell_price = client.get_order_book(symbol=symbol)['bids'][0][0]
        print('Sell @Market Price : ',float(sell_price),'Timestamp : ',str(datetime.now()))


Buy @Market Price :  57314.25 Timestamp :  2021-04-07 06:28:02.430658
Sell @Market Price :  57346.3 Timestamp :  2021-04-07 06:28:07.303302
Buy @Market Price :  57362.0 Timestamp :  2021-04-07 06:28:11.832747
Sell @Market Price :  57384.91 Timestamp :  2021-04-07 06:28:21.501294
Buy @Market Price :  57398.49 Timestamp :  2021-04-07 06:28:26.192400
